In [ ]:
### 주제: "산불 확산 패턴을 통한 피해 면적 예측 프로젝트"
# 핵심 키워드 정리
# 산불 확산 패턴 - 산불의 이동 및 확산 경로 분석
# 피해 면적 예측 - 산불로 인한 최종 피해 면적(km²)을 수치로 예측
# 위성 데이터 - NASA FIRMS(산불)와 POWER API(기상) 활용
# 머신러닝 모델	회귀 분석 기반 피해 예측 모델 개발

### 대략적 일정
## 1일차 (월요일)
# 오전 (4h): NASA FIRMS 데이터 수집 → 4명 분담: 위도/경도, brightness 등 확인 (pandas.read_csv)
#          : SUOMI VIIRS C2(VNP14IMG) + J1 VIIRS C2(VJ114IMG) 데이터 수집 → 위도/경도, FRP 확인.
# 오후 (4h): NASA POWER API 다운로드 → 기온, 풍속, 습도 체크 후 병합 시작 (pd.merge)
#          : NASA POWER API 기상 데이터 병합 → 기온, 풍속 등 체크.
# 화요일 대비 - GeoJSON 데이터 찾아보고 없으면 shapely(시각화에서 활용되는 라이브러리(사실 나도 잘 모름...))으로 다각형 생성 준비
# MODIS C6.1도 과거 참고용으로 다운로드 시작

## 2일차 (화요일)
# 오전 (4h): 데이터 전처리 → 결측치(평균값), 이상치 제거 (IQR)
# 오후 (4h): 탐색적 데이터 분석(EDA) + Folium → 상관관계 분석 (seaborn) 후 VIIRS C2로 산불 위치 맵 생성 (folium.Marker).
# Folium 사용 이유:
# 패턴 파악: 산불 분포를 지도로 시각화해 지역별 확산 경향 조기 확인
# 발표 준비: PPT용 기초 지도를 만들어 5일차 부담 줄임
# GPT 팁! - heatmap으로 brightness 강도 추가하면 패턴 더 선명해짐!

## 3일차 (수요일)
# 오전 (4h): 초기 모델 구축 → 선형 회귀, Ridge, Lasso 학습 (scikit-learn), RMSE/MAE 계산
# 오후 (4h): 피처 엔지니어링 → 풍속습도, 온도강수량 등 추가, 그래프 분석 (Folium 생략)
# 피처 엔지니어링 요약
#  - 데이터를 더 똑똑하게 가공해서 모델 성능 높이는 작업
#  - 풍속*습도, 고온 여부 등 산불 피해와 관련성 높은 변수 생성
#  - VIIRS의 FRP 활용해 피해 강도 변수 만들어보기

# 4일차 (목요일)
# 오전 (4h): 모델 개선 → Random Forest, XGBoost 등 추가 학습 및 튜닝 (GridSearchCV)
# 오후 (4h): 결과 분석 + Folium → VIIRS C2 최적 모델의 예측 피해 면적(km²)을 색상 원으로 맵에 추가 (folium.Circle)
# Folium 사용 이유: 최종 예측 결과를 시각화해 발표용 핵심 자료 완성, 모델별 성능 비교 가능
# 최고 모델의 예측만 빨간색으로 표시하면 시각적 임팩트 추가하기
# MODIS C6.1 예측과 비교 슬라이드 넣으면 깊이 추가

# 5일차 (금요일)
# 오전 (4h): PPT 완성 → Folium(VIIRS C2) 지도 2개(위치, 예측)와 모델 성능 표 삽입.
# 오후 (4h): 완성 되지 못한 부분 보완 및 PPT 발표 대본 및 연습


# PPT 구성
# 표지: "산불 피해 예측 프로젝트" – 팀명
# 소개: 산불 피해의 심각성 (통계 그래프).
# 데이터: NASA FIRMS + POWER API 개요, Folium 산불 위치 맵
# 방법: 전처리 → 다중 회귀 모델 → 검증 다이어그램
# 모델 비교: 선형 회귀, Random Forest 등 RMSE/MAE 표
# 결과: 예측 면적(km²) + Folium 피해 범위 맵 (4일차), 최적 모델 성능
# 결론: 기대 효과와 확장 가능성 (예: 실시간 예측)

###
# SUOMI VIIRS C2: 최신, 고해상도 산불 데이터의 주력
# J1 VIIRS C2   : SUOMI와 짝꿍으로 더 촘촘한 관측
# MODIS C6.1    : 과거를 돌아보는 데 유용한 백업

### SUOMI VIIRS C2 (Suomi NPP VIIRS Collection 2)
# SUOMI VIIRS는 Suomi National Polar-orbiting Partnership (Suomi NPP)라는 위성에 탑재된 VIIRS(Visible Infrared Imaging Radiometer Suite) 센서에서 나온 데이터
# 특징:
#  - 해상도     : 375m (산불 탐지에 더 세밀함).
#  - 관측 시간  : 하루에 오전 1:30쯤과 오후 1:30쯤, 지구를 두 번 커버
#  - 업데이트   : 최신 알고리즘으로 데이터를 재처리해서 더 정확함.
#  - 산불 데이터: NASA FIRMS에서 제공하는 "VNP14IMG"라는 이름으로 산불 위치와 강도(FRP: Fire Radiative Power)를 탐지
#  - 쉽게 말하면: Suomi NPP라는 위성이 찍은 최신 산불 사진첩

# J1 VIIRS C2 (NOAA-20 VIIRS Collection 2)
# J1 VIIRS는 NOAA-20 위성(원래 이름은 JPSS-1)에 탑재된 VIIRS 센서 데이터
# 특징:
#  - 해상도: SUOMI VIIRS와 동일한 375m.
#  - 관측 시간: 오전 2:20쯤과 오후 2:20쯤으로, SUOMI보다 50분 뒤에 지나감.
#  - 목적: SUOMI와 비슷하지만, 더 자주 관측해서 데이터 빈틈을 줄임.
#  - 산불 데이터: "VJ114IMG"라는 이름으로 FIRMS에서 제공. SUOMI와 거의 똑같은 정보를 주지만, 시간 차이 때문에 보완적인 역할.
#  - 쉽게 말하면: NOAA-20 위성이 SUOMI의 동생처럼 뒤따라가며 찍은 산불 사진첩

# MODIS C6.1 (MODIS Collection 6.1)
# MODIS는 Terra와 Aqua라는 두 위성에 탑재된 Moderate Resolution Imaging Spectroradiometer 센서의 데이터야. "C6.1"은 Collection 6.1, 즉 6번째 버전의 업데이트판.
# 특징:
# 해상도: 1km (VIIRS보다 덜 세밀함).
# 관측 시간: Terra는 오전 10:30쯤, Aqua는 오후 1:30쯤 지나감.
# 역사: 2000년대 초반부터 데이터를 쌓아왔으니 더 긴 시간 기록을 볼 수 있음.
# 산불 데이터: "MCD14"라는 이름으로 FIRMS에서 제공. 산불 위치와 FRP를 탐지하지만, 해상도가 낮아서 작은 불은 놓칠 수 있어.
# 쉽게 말하면: 오래된 형님 위성이 찍은 산불 사진첩의 6.1번째 버전이야. 좀 더 흐릿하지만 오래된 이야기를 들려줘!

# 차이점        비교
# 항목          SUOMI VIIRS C2      J1 VIIRS C2         MODIS C6.1
# 위성          Suomi NPP           NOAA-20 (JPSS-1)    Terra & Aqua
# 센서          VIIRS               VIIRS               MODIS
# 해상도        375m                375m                1km
# 관측 시간     1:30 AM/PM          2:20 AM/PM          10:30 AM, 1:30 PM
# 버전          Collection 2        Collection 2        Collection 6.1
# 장점          세밀하고 최신       SUOMI와 보완        긴 시간 기록
# 단점          짧은 역사           짧은 역사           덜 세밀함
# FIRMS 이름    VNP14IMG            VJ114IMG            MCD14

# SUOMI VIIRS C2와 J1 VIIRS C2: 최신이고 해상도가 높아서 산불 위치와 피해 면적 예측에 더 정확. 특히 375m 해상도는 작은 불도 잡아내니까 EDA와 Folium 시각화에 딱!
# MODIS C6.1: 과거 데이터(20년 이상)를 보고 싶을 때 유용해. 예를 들어, "과거 산불 패턴이 지금이랑 얼마나 달라졌나?" 같은 분석에 쓰면 좋음.
# 추천: 프로젝트에선 주로 VIIRS C2(SUOMI + J1)를 쓰고, 필요하면 MODIS C6.1로 과거 비교

In [ ]:
### 1. 산불 데이터 (fire_data)
# 필수 데이터 (NASA FIRMS 활용)
# fire_id	    str	        산불 고유 ID
# latitude	    float	    위도
# longitude	    float	    경도
# date	        str	        관측 날짜 (YYYY-MM-DD)
# brightness	float	    화재 밝기 지수 (K)
# confidence	int	        신뢰도 (0~100)
# frp	        float	    화재 방출력 (MW)

### 2. 기상 데이터 (weather_data)
# 필수 데이터 (NASA POWER API 활용)
# latitude	    float	    위도
# longitude	    float	    경도
# date	        str	        기상 데이터 날짜 (YYYY-MM-DD)
# temperature	float	    기온 (°C)
# humidity	    float	    상대 습도 (%)
# wind_speed	float	    풍속 (m/s)
# precipitation	float	    강수량 (mm)

### 3. 산불 확산 경계 데이터 (geojson_data)
# region_name	str	        지역명 (ex: California)
# geometry	    dict	    GeoJSON 형식의 다각형 좌표
# fire_count	int	        해당 지역에서 발생한 산불 수

# 산불 경계 데이터 (GeoJSON)
# GeoJSON을 찾아봐야 할 곳
# Global Fire Atlas: https://www.globalfiredata.org
# NASA FIRMS (GIS Data): https://firms.modaps.eosdis.nasa.gov/gisdata/
# OpenStreetMap Wildfire Data: https://overpass-turbo.eu
# 없으면 산불데이터로 형성 (open ai 질문 결과 shapely 라이브러리 활용, 산불 데이터 (latitude, longitude)를 Convex Hull 알고리즘을 이용해 다각형(Polygon)으로 변환

### 4. 예측 피해 면적 데이터 (pred_data)
# latitude	    float	    예측된 피해 중심 위도
# longitude	    float	    예측된 피해 중심 경도
# pred_area	    float	    예측된 피해 면적 (km²)
# confidence	float	    예측 신뢰도 (0~1)

# 공통 사용 데이터 변수명
# fire_data: NASA FIRMS 데이터
# weather_data: NASA POWER API 기상 데이터
# geojson_data: 산불 확산 경계 데이터 (GeoJSON)
# pred_data: 머신러닝 모델의 예측 결과

### 필요할거로 예상되는 파일 구조
# wildfire_project/
# - data/
#   - fire_data.csv  # NASA FIRMS 데이터
#   - weather_data.csv  # NASA POWER API 데이터
#   - fire_boundary.geojson  # 산불 확산 경계 (GeoJSON) (없으면 위도 경도 직접 활용해서 사용)

# - models/
#   - 우리가 배운 회귀 모델들 사용 후 선정 할 파일
#     (클래스로 만들어서 데이터 프레임으로 하거나 모델별로 파일 생성.) - 상의 후 결정

# - visualizations/
#   - fire_map.html  # Folium 지도 (산불 위치)
#   - fire_heatmap.html  # Folium 히트맵 (산불 강도)

# - main.ipynb  # 분석 코드
# - presentation.pptx  # 최종 발표 자료

In [1]:
import requests

# 기상 데이터 요청할 지역 (위도, 경도)
latitude = 53.7267  # 브리티시컬럼비아 근처
longitude = -127.6476

# 요청할 기간 (2015년 1월 1일 ~ 2024년 11월 30일)
start_date = "20150101"
end_date = "20250228"

# API 요청 URL
url = f"https://power.larc.nasa.gov/api/temporal/daily/point?parameters=T2M,WS2M,RH2M,PRECTOTCORR&community=AG&longitude={longitude}&latitude={latitude}&start={start_date}&end={end_date}&format=CSV"

# API 요청 보내기
response = requests.get(url)

# 파일 저장
if response.status_code == 200:
    with open("weather_data.csv", "w") as file:
        file.write(response.text)
    print("✅ 기상 데이터 다운로드 완료! (weather_data.csv)")
else:
    print("❌ 데이터 요청 실패!", response.status_code)


✅ 기상 데이터 다운로드 완료! (weather_data.csv)


In [2]:
import pandas as pd

In [3]:
file_path = "./weather_data.csv"

canada_org = pd.read_csv(file_path)
canada_org


#  원본 데이터 결측치 없음

,YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR
0,2015,1,-1.62,0.98,99.26,3.70
1,2015,2,-4.67,0.80,99.57,4.56
2,2015,3,-19.00,1.68,99.49,4.01
3,2015,4,-16.91,1.60,99.62,13.03
4,2015,5,-14.83,1.04,99.70,21.29
...,...,...,...,...,...,...
3707,2025,55,-0.97,0.59,95.83,2.70
3708,2025,56,-2.64,0.63,98.43,0.85
3709,2025,57,-1.32,0.64,98.46,5.43
3710,2025,58,-1.05,0.71,93.85,4.19


In [4]:
canada_org.tail()

,YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR
3707,2025,55,-0.97,0.59,95.83,2.70
3708,2025,56,-2.64,0.63,98.43,0.85
3709,2025,57,-1.32,0.64,98.46,5.43
3710,2025,58,-1.05,0.71,93.85,4.19
3711,2025,59,-1.95,0.45,94.49,2.45


In [5]:
canada_org.describe()

# 이상치 확인하기 이상치 없음

,YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR
count,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000,3712.000000
mean,2019.587823,180.716056,0.495148,0.611204,91.157982,4.150442
std,2.931968,106.386782,8.658110,0.325034,10.233032,5.334211
min,2015.000000,1.000000,-32.890000,0.150000,32.070000,0.000000
25%,2017.000000,87.000000,-4.450000,0.370000,85.210000,0.280000
50%,2020.000000,180.000000,-0.180000,0.540000,95.830000,2.190000
75%,2022.000000,273.000000,7.292500,0.760000,99.000000,6.110000
max,2025.000000,366.000000,24.890000,2.770000,100.000000,58.520000


In [6]:
canada_org.duplicated().value_counts()

# 중복치 없음

False    3712
Name: count, dtype: int64

In [7]:
# "DOY"컬럼의 날짜들을 "acq_date"로 0000-00-00형태로 변환시켰다.(윤년적용)
canada_org['acq_date'] = pd.to_datetime(canada_org['YEAR'].astype(str) + canada_org['DOY'].astype(str).str.zfill(3), format='%Y%j')

In [9]:
canada_org.head(50)

,YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR,acq_date
0,2015,1,-1.62,0.98,99.26,3.70,2015-01-01
1,2015,2,-4.67,0.80,99.57,4.56,2015-01-02
2,2015,3,-19.00,1.68,99.49,4.01,2015-01-03
3,2015,4,-16.91,1.60,99.62,13.03,2015-01-04
4,2015,5,-14.83,1.04,99.70,21.29,2015-01-05
5,2015,6,-8.92,1.03,100.00,14.60,2015-01-06
6,2015,7,-0.44,0.39,99.37,2.68,2015-01-07
7,2015,8,-8.44,1.46,99.79,0.10,2015-01-08
8,2015,9,-3.31,0.54,93.43,1.49,2015-01-09
9,2015,10,-3.78,0.66,99.55,0.85,2015-01-10


In [10]:
canada_org.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3712 entries, 0 to 3711
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   YEAR         3712 non-null   int64         
 1   DOY          3712 non-null   int64         
 2   T2M          3712 non-null   float64       
 3   WS2M         3712 non-null   float64       
 4   RH2M         3712 non-null   float64       
 5   PRECTOTCORR  3712 non-null   float64       
 6   acq_date     3712 non-null   datetime64[ns]
dtypes: datetime64[ns](1), float64(4), int64(2)
memory usage: 203.1 KB


In [11]:
# canada_nrt_sv 파일 불러들이기
file_path = "./data/fire_nrt_SV-C2_589621.csv"

canada_nrt_sv = pd.read_csv(file_path)
canada_nrt_sv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4386 entries, 0 to 4385
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    4386 non-null   float64
 1   longitude   4386 non-null   float64
 2   brightness  4386 non-null   float64
 3   scan        4386 non-null   float64
 4   track       4386 non-null   float64
 5   acq_date    4386 non-null   object 
 6   acq_time    4386 non-null   int64  
 7   satellite   4386 non-null   object 
 8   instrument  4386 non-null   object 
 9   confidence  4386 non-null   object 
 10  version     4386 non-null   object 
 11  bright_t31  4386 non-null   float64
 12  frp         4386 non-null   float64
 13  daynight    4386 non-null   object 
dtypes: float64(7), int64(1), object(6)
memory usage: 479.8+ KB


In [12]:
# canada_nrt_j1v 파일 불러들이기
file_path = "./data/fire_nrt_J1V-C2_589620.csv"

canada_nrt_j1v = pd.read_csv(file_path)
canada_nrt_j1v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5932 entries, 0 to 5931
Data columns (total 14 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   latitude    5932 non-null   float64
 1   longitude   5932 non-null   float64
 2   brightness  5932 non-null   float64
 3   scan        5932 non-null   float64
 4   track       5932 non-null   float64
 5   acq_date    5932 non-null   object 
 6   acq_time    5932 non-null   int64  
 7   satellite   5932 non-null   object 
 8   instrument  5932 non-null   object 
 9   confidence  5932 non-null   object 
 10  version     5932 non-null   object 
 11  bright_t31  5932 non-null   float64
 12  frp         5932 non-null   float64
 13  daynight    5932 non-null   object 
dtypes: float64(7), int64(1), object(6)
memory usage: 648.9+ KB


In [15]:
# canada_ac_sv 파일 불러들이기
file_path = "./data/fire_archive_SV-C2_589621.csv"

canada_ac_sv = pd.read_csv(file_path)
canada_ac_sv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4593709 entries, 0 to 4593708
Data columns (total 15 columns):
 #   Column      Dtype  
---  ------      -----  
 0   latitude    float64
 1   longitude   float64
 2   brightness  float64
 3   scan        float64
 4   track       float64
 5   acq_date    object 
 6   acq_time    int64  
 7   satellite   object 
 8   instrument  object 
 9   confidence  object 
 10  version     int64  
 11  bright_t31  float64
 12  frp         float64
 13  daynight    object 
 14  type        int64  
dtypes: float64(7), int64(3), object(5)
memory usage: 525.7+ MB


In [20]:
# canada_ac_j1v 파일 불러들이기
file_path = "./data/fire_archive_J1V-C2_589620.csv"

canada_ac_j1v = pd.read_csv(file_path)
canada_ac_j1v.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3684767 entries, 0 to 3684766
Data columns (total 15 columns):
 #   Column      Dtype  
---  ------      -----  
 0   latitude    float64
 1   longitude   float64
 2   brightness  float64
 3   scan        float64
 4   track       float64
 5   acq_date    object 
 6   acq_time    int64  
 7   satellite   object 
 8   instrument  object 
 9   confidence  object 
 10  version     int64  
 11  bright_t31  float64
 12  frp         float64
 13  daynight    object 
 14  type        int64  
dtypes: float64(7), int64(3), object(5)
memory usage: 421.7+ MB


In [21]:
# 4개의 산불데이터.csv 파일들을 모두 concat 함
canada_fire_data = pd.concat([canada_nrt_sv, canada_nrt_j1v, canada_ac_sv, canada_ac_j1v])

In [97]:
canada_fire_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,46.04363,-73.14433,297.48,0.51,0.41,2025-01-01,631.0,N,VIIRS,n,2.0NRT,268.65,1.55,N,NaN
1,55.64818,-112.29297,320.25,0.39,0.36,2025-01-01,951.0,N,VIIRS,n,2.0NRT,251.60,3.17,N,NaN
2,54.12277,-111.77706,323.20,0.40,0.37,2025-01-01,951.0,N,VIIRS,n,2.0NRT,250.17,2.60,N,NaN
3,56.24786,-120.41487,326.05,0.50,0.41,2025-01-01,951.0,N,VIIRS,n,2.0NRT,249.49,4.53,N,NaN
4,56.76768,-121.15940,321.27,0.54,0.42,2025-01-01,951.0,N,VIIRS,n,2.0NRT,253.00,2.89,N,NaN


In [22]:
canada_fire_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8288794 entries, 0 to 3684766
Data columns (total 15 columns):
 #   Column      Dtype  
---  ------      -----  
 0   latitude    float64
 1   longitude   float64
 2   brightness  float64
 3   scan        float64
 4   track       float64
 5   acq_date    object 
 6   acq_time    int64  
 7   satellite   object 
 8   instrument  object 
 9   confidence  object 
 10  version     object 
 11  bright_t31  float64
 12  frp         float64
 13  daynight    object 
 14  type        float64
dtypes: float64(8), int64(1), object(6)
memory usage: 1011.8+ MB


In [ ]:
# 이상치가 있어서 직접 .csv파일에서 찾아서 수정함
c= canada_fire_data['acq_date'] == "2018-04-006371"
canada_fire_data[c]

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,42.80264,-80.10513,295.36,0.56,0.51,2018-04-006371,NaN,N20,VIIRS,n,2,272.13,1.55,N,2.0


In [23]:
# canada_fire_data 데이터의 "acq_date"데이터의 타입을 to_datetime으로 변환(weather_data.csv와 merge하기 위함)
canada_fire_data['acq_date'] = pd.to_datetime(canada_fire_data['acq_date'])

In [24]:
canada_fire_data.head()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
0,46.04363,-73.14433,297.48,0.51,0.41,2025-01-01,631,N,VIIRS,n,2.0NRT,268.65,1.55,N,NaN
1,55.64818,-112.29297,320.25,0.39,0.36,2025-01-01,951,N,VIIRS,n,2.0NRT,251.60,3.17,N,NaN
2,54.12277,-111.77706,323.20,0.40,0.37,2025-01-01,951,N,VIIRS,n,2.0NRT,250.17,2.60,N,NaN
3,56.24786,-120.41487,326.05,0.50,0.41,2025-01-01,951,N,VIIRS,n,2.0NRT,249.49,4.53,N,NaN
4,56.76768,-121.15940,321.27,0.54,0.42,2025-01-01,951,N,VIIRS,n,2.0NRT,253.00,2.89,N,NaN


In [25]:
canada_fire_data.tail()

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,version,bright_t31,frp,daynight,type
3684762,43.26424,-79.80718,299.70,0.50,0.41,2024-11-30,655,N20,VIIRS,n,2,268.02,2.07,N,3.0
3684763,42.94913,-82.42247,298.19,0.43,0.46,2024-11-30,655,N20,VIIRS,n,2,270.40,1.16,N,2.0
3684764,42.94436,-82.41984,298.87,0.43,0.46,2024-11-30,655,N20,VIIRS,n,2,266.83,1.49,N,2.0
3684765,50.13165,-95.68285,322.04,0.39,0.36,2024-11-30,834,N20,VIIRS,n,2,256.36,2.04,N,0.0
3684766,56.76218,-117.61443,322.75,0.39,0.36,2024-11-30,1014,N20,VIIRS,n,2,246.62,2.32,N,0.0


In [26]:
canada_merged_data = pd.merge(canada_fire_data, canada_org, on='acq_date', how='left')

In [41]:
canada_merged_data.tail(20)

,latitude,longitude,brightness,scan,track,acq_date,acq_time,satellite,instrument,confidence,...,bright_t31,frp,daynight,type,YEAR,DOY,T2M,WS2M,RH2M,PRECTOTCORR
4375,42.80462,-80.09727,305.28,0.44,0.38,2025-03-15,703,N,VIIRS,n,...,277.95,2.45,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4376,42.31218,-82.05047,299.65,0.51,0.41,2025-03-15,703,N,VIIRS,n,...,273.77,0.76,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4377,54.36203,-111.18905,323.25,0.33,0.55,2025-03-15,841,N,VIIRS,n,...,246.33,2.86,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10310,49.01395,-102.47881,323.21,0.42,0.38,2025-03-15,905,N20,VIIRS,n,...,265.73,2.77,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4378,48.25239,-79.02308,316.87,0.66,0.73,2025-03-15,841,N,VIIRS,n,...,266.74,2.98,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4380,48.66843,-91.27053,304.04,0.40,0.45,2025-03-15,843,N,VIIRS,n,...,275.38,1.49,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4381,50.51693,-104.62907,328.05,0.50,0.41,2025-03-15,843,N,VIIRS,n,...,263.08,4.93,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4382,49.01464,-102.48147,321.07,0.73,0.76,2025-03-15,1024,N,VIIRS,n,...,260.97,5.58,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4383,49.01448,-102.47631,341.31,0.73,0.76,2025-03-15,1024,N,VIIRS,n,...,261.97,6.69,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10288,43.07022,-80.85635,309.77,0.40,0.37,2025-03-15,726,N20,VIIRS,n,...,272.67,1.13,N,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
canada_merged_data = canada_merged_data.sort_values(by='acq_date')

In [30]:
canada_merged_data.to_csv("canada_merged_data", index=False)

### 통합데이터 불러들이기

In [32]:
canada_all = pd.read_csv("./canada_merged_data.csv")

C:\Users\Administrator\AppData\Local\Temp\ipykernel_9400\3568184484.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  canada_all = pd.read_csv("./canada_merged_data.csv")


In [ ]:
print(canada_all[canada_all.isnull().any(axis=1)])


         latitude  longitude  brightness  scan  track    acq_date  acq_time  \
8276649  42.80925  -80.10257      298.66  0.56   0.52  2024-12-01       635   
8276650  56.40762 -119.07354      327.42  0.47   0.64  2024-12-01      1136   
8276651  56.41240 -119.06723      349.01  0.47   0.64  2024-12-01      1136   
8276652  56.41238 -119.06930      351.16  0.47   0.64  2024-12-01      1136   
8276653  57.00581 -111.47849      335.82  0.75   0.77  2024-12-01      1134   
...           ...        ...         ...   ...    ...         ...       ...   
8288789  42.80766  -80.09614      302.45  0.41   0.37  2025-03-15       726   
8288790  49.85670 -119.13495      326.70  0.49   0.40  2025-03-16      2017   
8288791  49.99203 -119.50257      331.92  0.57   0.52  2025-03-16      1951   
8288792  50.98280 -105.09577      324.37  0.46   0.39  2025-03-16       845   
8288793  48.66678  -91.25076      328.35  0.43   0.38  2025-03-16      1834   

        satellite instrument confidence  ... bright

In [ ]:
# 결측치를 확인해본결과
# type, YEAR, DOY, T2M, WS2M, RH2M, PRECTOTCORR
# 
canada_all.isnull().sum(axis=0)

latitude           0
longitude          0
brightness         0
scan               0
track              0
acq_date           0
acq_time           0
satellite          0
instrument         0
confidence         0
version            0
bright_t31         0
frp                0
daynight           0
type           10318
YEAR            1292
DOY             1292
T2M             1292
WS2M            1292
RH2M            1292
PRECTOTCORR     1292
dtype: int64